# Depth

By **Franklin Oliveira**

-----

This notebook outputs Depth charts for the `poliqueta` collection. Database is in <font color='blue'>'IBUFRJ27.07.2020 - visualização.xlsx'</font> and <font color='blue'>'MNRJP27.07.2020 - visualização.xls'</font>.

In [61]:
import datetime
import numpy as np
import pandas as pd

from collections import defaultdict

# pacotes para visualização rápida
import seaborn as sns
import matplotlib.pyplot as plt

# pacote para visualização principal
import altair as alt

# habilitando renderizador para notebook
# alt.renderers.enable('notebook')
alt.renderers.enable('default')


# desabilitando limite de linhas
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

## Importing data...

Importing pre-treated data in `1-data_treatment.ipynb`. In this notebook, I'm doing only some minnor adjustments for visualization purposes only. For a full traceback of data treatment, please see the `1-data_treatment` notebook.

In [62]:
# treated_db was the previous db I used to write all this code
NewTable = pd.read_csv('./data/merged_db.csv', sep=';', encoding='utf-8')

In [63]:
# formatando a string NaN
NewTable['family'] = NewTable['family'].apply(lambda x: 'NaN' if x=='Nan' else x)

<br>

<font size=5>**Color Palette per Order**</font>

These images were used as inspiration (https://color.adobe.com/create/image)


<div class='row' style='padding-top:20px;'>
    <div class='col-md-6'>
        <img src="./src/img1.jpg" width='400px'>
    </div>
    <div class='col-md-6'>
        <img src="./src/img2.jpg" width='400px'>
    </div>
</div>

<br>

A partir das imagens acima, selecionamos cores (centróides) para criar a paleta de cores. Foram elas: 
<ul>
    <li style='color:#3CA67F'><b> #3CA67F </b># verde</li>
    <li style='color:#7A9FBF'><b> #7A9FBF </b># azul</li>
    <li style='color:#D94814'><b> #D94814 </b># laranja</li>
    <li style='color:#D96236'><b> #D96236 </b># laranja 2</li>
    <li style='color:#F2B999'><b> #F2B999 </b># 'cor de pele'</li>
    <li style='color:#A66C4B'><b> #A66C4B </b># marrom 1</li>
    <li style='color:#732C02'><b> #732C02 </b># marrom 2</li>
</ul>

A partir das cores "centróides", utilizamos a ferramenta Color Crafter para selecionar diferentes "shades" e auxiliar para categorização em diferentes grupos sugeridos pela equipe de Poliquetas do Museu Nacional. 

<ul>
    <li style='color:#3CA67F'><b> Verde: </b> ['#daffef', '#bbebd3', '#9adabc', '#77c8a5', '#57b791', '#3ca67f', '#2a9670', '#238762', '#257a56']</li>
    <li style='color:#7A9FBF'><b> Azul: </b> ['#e7e5df', '#ccd2d8', '#b2c0d0', '#96afc8', '#7a9fbf', '#5d90b6', '#3c81ae', '#0673a4', '#00669a']</li>
    <li style='color:#D94814'><b> laranja: </b> ['#ffbd84', '#ffaa74', '#ff9760', '#ff814b', '#fc6b36', '#eb5824', '#d94814', '#c83b03', '#b73000']</li>
    <li style='color:#D96236'><b> laranja 2: ['#ffeba9', '#ffd391', '#ffbb7b', '#fda468', '#f18e56', '#e57846', '#d96236', '#cc4d28', '#bf381b']</b> </li>
    <li style='color:#F2B999'><b> cor de pele: ['#ffe9c3', '#fbd0ad', '#f2b999', '#e8a287', '#dd8c76', '#d27666', '#c76158', '#bb4d4b', '#ae393e']</b> </li>
    <li style='color:#A66C4B'><b> marrom 1: ['#d9c6af', '#ccad96', '#c1977c', '#b48061', '#a66c4b', '#975b39', '#874c2c', '#774124', '#683720']</b> </li>
    <li style='color:#732C02'><b> marrom 2: ['#eebd93', '#dfa47a', '#d28d60', '#c37746', '#b4622f', '#a3501d', '#92420e', '#823606', '#732c02']</b> </li>
</ul>



**Colors  (antigas):** 

<ul>
    <li style='color:#41A681'><b> #41A681 </b># verde1</li>
    <li style='color:#3CA67F'><b> #3CA67F </b># verde2</li>
    <li style='color:#7ACAAB'><b> #7ACAAB </b># verde claro</li>
    <li style='color:#78a1a1'><b> #78a1a1 </b># azul</li>
    <li style='color:#8ABFB0'><b> #8ABFB0 </b># azul claro</li>
    <li style='color:#FFB27C'><b> #FFB27C </b># cor de pele clara</li>
    <li style='color:#F29877'><b> #F29877 </b># cor de pele</li>
    <li style='color:#ed845e'><b> #ed845e </b># laranja claro1</li>
    <li style='color:#D96236'><b> #D96236 </b># laranja claro2</li>
    <li style='color:#D95323'><b> #D95323 </b># laranja 1</li>
    <li style='color:#D94B18'><b> #D94B18 </b># laranja 2</li>
    <li style='color:#D9C2AD'><b> #D9C2AD </b># bege</li>
    <li style='color:#A66C4B'><b> #A66C4B </b># marrom claro</li>
    <li style='color:#86471B'><b> #86471B </b># marrom1</li>
    <li style='color:#732C02'><b> #732C02 </b># marrom2</li>
    <li style='color:#592202'><b> #592202 </b># marrom escuro1</li>
    <li style='color:#3D1806'><b> #3D1806 </b># marrom escuro2</li>
    <li style='color:#0D0D0D'><b> #0D0D0D </b># preto</li>
</ul>



In [64]:
# 242 NaNs
# NewTable['order'].value_counts(dropna=False)

In [65]:
# determinando cores de acordo com a planilha (2020.10.01 - IB e MN - Cores visualização.xlsx)
ordens = NewTable['order'].unique()
familias = NewTable['family'].unique()

# # o agrupamento é feito por famílias (ordem daquelas famílias deve assumir certa cor)
cores_ordem = {
    'Spionida':'#41A681',   # verde
    'Sabellida':'#7ACAAB',  # verde claro
    'Canalipalpata':'#78a1a1',  # azul
    'Amphinomida':'#8ABFB0',  # azul claro
    'Eunicida':'#A66C4B', # marrom claro
    'Phyllodocida':'#732C02', # marrom2
    'Terebellida':'#ed845e', # laranja claro1
    'Scolecida':'#D94B18', # laranja 2
    np.NAN:'#0D0D0D',  # preto
    
    # ordens não citadas na planilha:
    'Sipuncula':'#D9C2AD', # bege
    'Crassiclitellata':'#FFB27C', # cor de pele clara
    'Aspidosiphonida':'#F29877',  # cor de pele
    
}

# paleta de cores por família
cores_familia = {
    'Magelonidae':'#238762',    # verde escuro 
    'Oweniidae':'#3CA67F',      # verde (centroide)  
    'Chaetopteridae':'#77c8a5', # verde
    'Amphinomidae':'#bbebd3',   # verde claro
    'Lumbrineridae':'#e7e5df',  # azul claro 1
    'Dorvilleidae':'#b2c0d0',   # azul claro2
    'Oenonidae':'#7A9FBF',      # azul (centroide)
    'Eunicidae':'#3c81ae',      # azul
    'Onuphidae':'#00669a',      # azul escuro
    'Syllidae':'#ffbd84', 
    'Typhloscolecidae':'#ffaa74', 
    'Aphroditidae':'#ff9760', 
    'Acoetidae':'#ff814b', 
    'Chrysopetalidae':'#fc6b36', 
    'Eulepethidae':'#eb5824',
    'Lopadorrhynchidae':'#d94814',  # laranja (centroide)
    'Polynoidae':'#c83b03',
    'Nereididae':'#b73000',
    'Nephtyidae':'#f18e56',
    'Glyceridae':'#D96236',         # laranja 2 (centroide)
    'Tomopteridae':'#bf381b',
    'Serpulidae':'#fbd0ad',
    'Sabellidae':'#f2b999', # cor de pele (centroide)
    'Sabellariidae':'#e8a287',
    'Spionidae':'#d27666',
    'Ampharetidae':'#b48061',
    'Pectinariidae':'#a66c4b',  # marrom 1 (centroide),
    'Trichobranchidae':'#975b39',
    'Terebellidae':'#874c2c',
    'Cirratulidae':'#774124',
    'Flabelligeridae':'#683720',
    'Sternaspidae':'#eebd93',
    'Orbiniidae':'#dfa47a',
    'Opheliidae':'#d28d60',
    'Capitellidae':'#c37746',
    'Arenicolidae':'#b4622f',
    'Cossuridae':'#a3501d',
    'Scalibregmatidae':'#92420e',
    'Paraonidae':'#823606',
    'Maldanidae':'#732c02', # marrom 2 (centroide)
    'NaN':'#0D0D0D',  # preto
}

In [66]:
# Lista de Famílias que não estão com cor atribuída

for f in NewTable['family'].unique():
    if f not in list(cores_familia.keys()) and f != 'Nan':
        print('-', f)

- Phyllodocidae
- Goniadidae
- Pilargidae
- Poecilochaetidae
- Sigalionidae
- Hesionidae
- Longosomatidae
- Euphrosinidae
- Sphaerodoridae
- Apistobranchidae
- Pholoidae
- Sipunculidae
- Alciopidae
- Pontodoridae
- Iospilidae
- Megascolecidae
- Lacydoniidae
- Aspidosiphonidae
- Trochochaetidae


<br>

---

## Graphs

### Total amount of catalogations per year

x: Start Year (from Start Date)
y: number of catalogations per year

In [67]:
# counting catalog. per year
teste = NewTable['start_year'].value_counts()
teste = teste.reset_index().rename(columns={'index':'year', 'start_year':'counts'})

In [68]:
# adjusting columns for graphs
teste['year'] = teste['year'].apply(lambda x:str(x).split('/')[0].split('.')[0]).astype(int)
teste = teste.groupby('year').sum().reset_index() # soma do total de bichos coletados por ano

In [69]:
# min e max para eixo X (year)
min_x = teste['year'].min()
max_x = teste['year'].max()

# taking natural log
teste['ln_counts'] = teste['counts']

<font color='red'>**ideia:** fazer crescimento em log (referencias em Bio. fazem isso)</font>

In [70]:
temp = alt.Chart(data= teste, width=800, title= 'Collected animals per year').mark_bar().encode(
    x= alt.X('year', type='ordinal', title='Sampling Year'),
    y= alt.Y('ln_counts', type='quantitative', title='Contagem')
)

# temp.save('./graphs/coletas_por_ano.html')
# temp

In [71]:
# counting catalog. per year
# teste = NewTable['cataloged_year'].value_counts()
# teste = teste.reset_index().rename(columns={'index':'year', 'cataloged_year':'counts'})

teste = NewTable.groupby(['cataloged_year', 'collection_prefix']).count()
teste = teste['catalog_number'].reset_index().rename(columns={'catalog_number':'counts'})

# min e max para eixo X (year)
min_x = teste['cataloged_year'].min()
max_x = teste['cataloged_year'].max()

In [72]:
temp = alt.Chart(data= teste, width=800, title= 'Cataloged animals per year').mark_bar().encode(
    x= alt.X('cataloged_year', type='ordinal', title='Cataloged Year'),
    y= alt.Y('counts', type='quantitative', title='Contagem'), 
    color= alt.Color('collection_prefix', title='Coleção')
)

temp = temp.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# temp.save('./graphs/catalogacoes_por_ano.html')

temp

alt.Chart(...)

-----

## Depth per family

In [73]:
# subsetting
teste = NewTable[['min_depth','family','order', 'start_year', 'qualifier', 'catalog_number', 
                  'genus', 'species', 'collector_full_name', 'country','state','locality', 'type']].copy()

# sorting
teste = teste.sort_values(['min_depth','family'])

# dropping na
teste.dropna(subset=['min_depth'], inplace=True)

# making sure altitude is a floating point number
teste['min_depth'] = teste['min_depth'].astype(float)

# extremes for scale
max_y = teste['min_depth'].max()
min_y = teste['min_depth'].min()

In [74]:
# escala customizada para o eixo Y
custom_y_scale = []

i = 0

while i < max_y+1:
    if i % 20 == 0 and i <= 200:
        custom_y_scale.append(i)
    elif i % 100 == 0 and i <=500:
        custom_y_scale.append(i)
    elif i % 500 == 0:
        custom_y_scale.append(i)
    i+=1
    
custom_y_scale.append(custom_y_scale[-1] + 500)

# custom_y_scale

In [78]:
temp = alt.Chart(teste, title='Depth per family', width=800, height=1200).mark_circle().encode(
    x = alt.X('family', type='nominal', title='Family', 
              sort= alt.EncodingSortField('min_depth', op='max', order='ascending')),
    y = alt.Y('min_depth', type='quantitative', title='Depth (in meters)',
              scale = alt.Scale(domain=[max_y, min_y]),
              axis= alt.Axis(values=custom_y_scale, tickCount=len(custom_y_scale))),
    color= alt.Color('family', title='Family', 
                     scale=alt.Scale(domain=list(cores_familia.keys()), range=list(cores_familia.values())),
                     legend= alt.Legend(columns=2, symbolLimit=42)),
    tooltip = alt.Tooltip(['catalog_number', 'family','genus','species', 'type',
                            'qualifier', 'start_year','collector_full_name',
                            'country', 'state', 'locality', 'min_depth'])
)

temp = temp.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# temp.save('./graphs/depth/depth_per_family-custom_scale.html')
# temp.facet(row='order').save('./graphs/depth/depth_per_family-facetado.html')

# temp

In [79]:
data = teste.copy()
data['type'] = data['type'].apply(lambda x:'NaN' if str(x).lower() == 'nan' else x)

select_family = alt.selection_multi(fields=['family'], bind='legend')
select_type = alt.selection_multi(fields=['type'], bind='legend')

### TESTANDO shape com tipos
temp = alt.Chart(data, title='Depth per family', width=800).mark_point(filled=True).encode(
    x = alt.X('family', type='nominal', title='Family', 
              sort= alt.EncodingSortField('min_depth', op='max', order='ascending')),
    y = alt.Y('min_depth', type='quantitative', title='Depth (in meters)',
              scale = alt.Scale(domain=[max_y, min_y])),
    color= alt.Color('family', title='Family', 
                     scale=alt.Scale(domain=list(cores_familia.keys()), range=list(cores_familia.values())),
                     legend= alt.Legend(columns=2, symbolLimit=42)),
    shape= alt.Shape('type:N', title='Type', 
                    scale= alt.Scale(domain=['Holotype', 'Neotype','Paratype', 'NaN'],
                                     range=['triangle', 'square', 'cross','circle'])),
    opacity= alt.condition(select_type, alt.value(1), alt.value(0)),
    tooltip = alt.Tooltip(['catalog_number', 'family','genus','species', 'type',
                            'qualifier', 'start_year','collector_full_name',
                            'country', 'state', 'locality', 'min_depth'])
).add_selection(select_family, select_type).transform_filter(select_family)

temp = temp.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# temp.save('./graphs/depth/depth_per_family-shape_tipos.html')
# temp

<br>

## Depth per genus

In [52]:
teste = NewTable[['min_depth','family','order', 'start_year', 'qualifier', 'catalog_number', 
                  'genus', 'species',  'collector_full_name',
                            'country', 'state', 'locality']].copy()

# making sure altitude is a floating point number
teste['min_depth'] = teste['min_depth'].astype(float)
teste['genus'] = teste['genus'].str.capitalize()

# sorting
teste = teste.sort_values(['min_depth','genus'])

# dropping na
teste = teste.dropna(subset=['min_depth'])

# extremes for y axis
max_y = teste['min_depth'].max()
min_y = teste['min_depth'].min()

In [60]:
# ordering x-axis per mean altitude - OUTLIER: ordem nula
temp = alt.Chart(teste, title='Depth per genus',
                width= 1500).mark_circle().encode(
    x = alt.X('genus', type='nominal', title='Genus',
             sort=alt.EncodingSortField('min_depth', op="max", order="ascending")),
    y = alt.Y('min_depth', type='quantitative', title='Depth (in meters)',
              scale= alt.Scale(domain= [max_y, min_y])),
    color = alt.Color('family:N', title='Family',
                      scale= alt.Scale(domain=list(cores_familia.keys()), range=list(cores_familia.values())),
                      legend= alt.Legend(columns=2, symbolLimit=42)),
    tooltip = alt.Tooltip(['catalog_number', 'family', 'genus','species',
                            'qualifier', 'start_year','collector_full_name',
                            'country', 'state', 'locality', 'min_depth'])
)

temp = temp.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12,
#     orient='bottom'
)

# temp.save('./graphs/depth/genus/depth-per-genus.html')
# temp

<br>

## Separating per Min Depth

<font color='red' size=4>Separando grupos de "maior profundidade"</font>


In [48]:
genus = teste['genus'].unique()

d = defaultdict()

for gen in genus:
    depth = teste[teste['genus'] == gen]['min_depth'].max()
    d[gen] = depth
        
d = pd.DataFrame(dict(d), index=[0]).transpose().reset_index()
d.columns = ['genus', 'max_depth']

In [49]:
# divisão entre marés (sugerido pelo pessoal do Museu)
threshold = 200

# maior profundidade (>= 500m)
grupo1 = d[d['max_depth'] > threshold]['genus']

# menor profundidade
grupo2 = d[d['max_depth'] <= threshold]['genus']

#### grupo de maior profundidade

In [50]:
# ordering x-axis per mean altitude
temp = alt.Chart(teste[teste['genus'].isin(grupo1)], width= 800,
                 title='Depth per genus').mark_circle().encode(
    x = alt.X('genus', type='nominal', title='Genus',
             sort=alt.EncodingSortField('min_depth', op='max', order="ascending")),
    y = alt.Y('min_depth', type='quantitative', title='Depth (in meters)', 
              scale= alt.Scale(domain=[max_y, 0])),
    color = alt.Color('family:N', title='Family',
                      scale= alt.Scale(domain=list(cores_familia.keys()), range=list(cores_familia.values())),
                      legend= alt.Legend(columns=2, symbolLimit=42)),
    tooltip = alt.Tooltip(['catalog_number', 'family', 'genus','species',
                            'qualifier', 'start_year','collector_full_name',
                            'country', 'state', 'locality', 'min_depth'])
)

temp = temp.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# temp.save('./graphs/depth/genus/depth_per_genus-maior-profundidade.html')
# temp

#### grupo de menor profundidade

menos pontos (generos menos representados)

In [51]:
# ordering x-axis per mean altitude
temp = alt.Chart(teste[teste['genus'].isin(grupo2)], width= 800,
                 title='Depth per genus').mark_circle().encode(
    x = alt.X('genus', type='nominal', title='Genus',
             sort=alt.EncodingSortField('min_depth', op='max', order="ascending")),
    y = alt.Y('min_depth', type='quantitative', title='Depth (in meters)', 
              scale= alt.Scale(domain=[threshold, 0])),
     color = alt.Color('family:N', title='Family',
                      scale= alt.Scale(domain=list(cores_familia.keys()), range=list(cores_familia.values())),
                      legend= alt.Legend(columns=2, symbolLimit=42)),
    tooltip = alt.Tooltip(['catalog_number', 'family', 'genus','species',
                            'qualifier', 'start_year','collector_full_name',
                            'country', 'state', 'locality', 'min_depth'])
)

temp = temp.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# temp.save('./graphs/depth/genus/depth_per_genus-menor-profundidade.html')
# temp

<br>

**Thats it!**

-----